In [10]:
import xpress as xp
import pandas as pd
import numpy as np
import math

In [11]:
# VERY_LARGE_NUMBER = 10000
lambda1 = 1
lambda2 = 10000*lambda1


model = xp.problem()
model.controls.maxtime = 600 #max search time
df = pd.read_excel("person.xlsx")

In [12]:
#person info 
num_guests = len(df)

In [13]:

matric = df['Matric'].tolist()
for index,value in enumerate(matric):
    if not math.isnan(value):
        matric[index] = int(matric[index]) 

gender = df['Gender'].tolist()
for index,value in enumerate(gender):
    if value == 'M':
        gender[index] = 1
    elif value == "F":
        gender[index] = -1
    else: # None
        gender[index] = 0

In [14]:
ppl_type = df['TYPE'].tolist()
NoA = 0
NoS = 0
NoF = 0
NoP = len(ppl_type)

is_ALUMNI = []
is_STUDENT = []
is_FELLOW = []
for index,value in enumerate(ppl_type):
    if value == 'ALUMNI':
        is_ALUMNI.append(1)
        is_STUDENT.append(0)
        is_FELLOW.append(0)
        NoA += 1
    elif value == 'STUDENT':
        is_ALUMNI.append(0)
        is_STUDENT.append(1)
        is_FELLOW.append(0)
        NoS += 1
    elif value == 'FELLOW':
        is_ALUMNI.append(0)
        is_STUDENT.append(0)
        is_FELLOW.append(1)
        NoF += 1


# high table idx
notes = df['Notes'].tolist()
to_high_table_id = []
for index,value in enumerate(notes):
    if type(value) == str:
        to_high_table_id.append(1)
    else:
        to_high_table_id.append(0)
# table info

num_tables = 6
table_seats = [ 21, 16, 26, 10, 20, 17 ]
high_table_id = [ 1, 0, 0, 0, 0, 0 ]


In [15]:

# NoA constrants for each table
cons_NoA = [NoA/NoP * table_seats[i] +2 for i in range(len(table_seats))]
# NoS constrants for each table
cons_NoS = [NoS/NoP * table_seats[i] +2 for i in range(len(table_seats))]
# NoF constrants for each table
cons_NoF = [NoF/NoP * table_seats[i] +2 for i in range(len(table_seats))]

maxseats = sum(table_seats)

In [16]:
# decision variables

x=np.array([[xp.var(name=f'x{i}_{j}',vartype=xp.binary) for j in range(num_tables)] for i in range(num_guests)],dtype=xp.npvar)
gender_diff = np.array([xp.var(name=f'gender_diff{i}',vartype=xp.binary) for i in range(num_tables)],dtype=xp.npvar)
max_matric = np.array([xp.var(name = f'max_matric{i}',vartype=xp.integer) for i in range(num_tables)],dtype=xp.npvar)
min_matric = np.array([xp.var(name = f'min_matric{i}',vartype=xp.integer) for i in range(num_tables)],dtype=xp.npvar)
# max_matric = [model.addVariable() for i in range(num_tables)]
# min_matric = [model.addVariable() for i in range(num_tables)]

model.addVariable(x)
model.addVariable(gender_diff)
model.addVariable(max_matric)
model.addVariable(min_matric)


In [17]:
# constrants

# 每个人有且只有一个位置

for i in range(num_guests):
    model.addConstraint(xp.Sum(x[i, j] for j in range(num_tables)) == 1)

# 每个桌子座位数量=人数
for i in range(num_tables):
       model.addConstraint(xp.Sum(x[j, i] for j in range(num_guests)) == table_seats[i])

# # # 特定的人坐在主桌
for i in range(len(to_high_table_id)):
    if to_high_table_id[i] == 1:
        for j in range(len(high_table_id)):
            if high_table_id[j] == 1:
                model.addConstraint(x[i,j] == 1)

# 每个桌子的身份数量约束
for i in range(len(table_seats)):
    model.addConstraint(xp.Sum(x[j,i] * is_ALUMNI[j] for j in range(num_guests)) <= cons_NoA[i])
    model.addConstraint(xp.Sum(x[j,i] * is_STUDENT[j] for j in range(num_guests)) <= cons_NoS[i])
    model.addConstraint(xp.Sum(x[j,i] * is_FELLOW[j] for j in range(num_guests)) <= cons_NoF[i])

for i in range(NoP):
    if not math.isnan(matric[i]):
        for jj in range(num_tables): 
            model.addConstraint(max_matric[jj] >= x[i,jj]*matric[i] - 10000*(1-x[i,jj]))
            model.addConstraint(min_matric[jj] <= x[i,jj]*matric[i] + 10000 * (1-x[i,jj]))

#gender_diff constraints  abs constraint
for j in range(num_tables):
    model.addConstraint(gender_diff[j] >= xp.Sum(x[i,j] * gender[i] for i in range(num_guests)))
    model.addConstraint(gender_diff[j] >= -xp.Sum(x[i,j] * gender[i] for i in range(num_guests)))


In [18]:
# objective func

range_matric = xp.Sum(max_matric[j] - min_matric[j] for j in range(num_tables))
sum_gender_diff = xp.Sum(gender_diff[j] for j in range(num_tables))


model.setObjective(lambda1*sum_gender_diff+lambda2*range_matric)
# model.addObjective(range_matric)
# model.addobj(gender_diff, [1.0]*len(gender_diff))

model.optimize()
res = model.getSolution(x)


for j in range(num_tables):
    out = ""
    out += f"table {j+1}:\n"
    
    guest_id = [i for i in range(num_guests) if res[i,j] == 1]
    out += f"guests: {[i+1 for i in guest_id]}\n"

    num_male = sum([1 if gender[i] == 1 else 0 for i in guest_id])
    num_female = sum([1 if gender[i] == -1 else 0 for i in guest_id])
    num_unknown_gender = sum([1 if gender[i] == 0 else 0 for i in guest_id])
    out += f"male: {num_male}, female: {num_female}, unknown: {num_unknown_gender}\n"
    num_alumni = sum([1 if is_ALUMNI[i] == 1 else 0 for i in guest_id])
    num_student = sum([1 if is_STUDENT[i] == 1 else 0 for i in guest_id])
    num_fellow = sum([1 if is_FELLOW[i] == 1 else 0 for i in guest_id])
    out += f"alumni: {num_alumni}, student: {num_student}, fellow: {num_fellow}\n"

    matric_info = [matric[i] for i in guest_id]
    out += f"matric info: {matric_info}\n"
    score_gender_diff = abs(num_female-num_male)
    out += f"gender diff: {score_gender_diff}\n"
    score_matric_diff = max(matric_info) - min(matric_info)
    out += f"matric diff: {score_matric_diff}\n"
    out+= '\n'
    print(out)

FICO Xpress v9.2.4, Hyper, solve started 2:16:40, Jul 24, 2024
Heap usage: 956KB (peak 1164KB, 768KB system)
Minimizing MILP noname using up to 8 threads and up to 8006MB memory, with these control settings:
MAXTIME = 600
OUTPUTLOG = 1
Original problem has:
      1465 rows          678 cols         5111 elements       678 entities
Presolved problem has:
      1128 rows          612 cols         4200 elements       612 entities
LP relaxation tightened
Presolve finished in 0 seconds
Heap usage: 1416KB (peak 2358KB, 768KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.20e+04] / [ 2.44e-04,  1.97e+00]
  RHS and bounds [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
  Objective      [min,max] : [ 1.00e+00,  1.00e+04] / [ 1.00e+00,  1.00e+04]
Autoscaling applied standard scaling

Symmetric problem: generators: 41, support set: 402
 Number of orbits: 156, largest orbit: 8
 Row orbits: 288, row suppo

    1454  1790005.000 -302078179.8     15    977     99 3.04e+08      81     10
    1557  1790005.000 -302078179.8     15   1149     44 3.04e+08     314     10
    1658  1790005.000 -302078179.8     15   1235     48 3.04e+08     274     11
    1760  1790005.000 -302078179.8     15   1235     36 3.04e+08     302     11
    1867  1790005.000 -302078179.8     15   1283     72 3.04e+08     164     11
    1970  1790005.000 -302078179.8     15   1307     66 3.04e+08     208     11
Resetting tree to root.

Performing root presolve...

Reduced problem has:    1547 rows     612 columns     13319 elements
Presolve dropped   :       0 rows       0 columns        35 elements
Presolve tightened :       294 elements
Symmetric problem: generators: 41, support set: 402
 Number of orbits: 156, largest orbit: 8
 Row orbits: 288, row support: 744
Will try to keep branch and bound tree memory usage below 492MB
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
384195      -479910000.0    

    4394  1570001.000 -302078179.8     25   1805     79 3.04e+08     182     31
    4496  1570001.000 -302078179.8     25   1805     51 3.04e+08     112     31
    4611  1570001.000 -302078179.8     25   1820    156 3.04e+08      63     32
    4727  1570001.000 -302078179.8     25   2013     92 3.04e+08     114     32
    4838  1570001.000 -302078179.8     25   2138    112 3.04e+08     114     32
    4939  1570001.000 -302078179.8     25   2218    140 3.04e+08     109     33
    5056  1570001.000 -302078179.8     25   2349    154 3.04e+08      72     33
    5169  1570001.000 -302078179.8     25   2556     94 3.04e+08     146     34
    5269  1570001.000 -302078179.8     25   2557     48 3.04e+08     279     34
    5380  1570001.000 -302078179.8     25   2688    157 3.04e+08      56     35
    5482  1570001.000 -302078179.8     25   2751     36 3.04e+08     290     35
    5582  1570001.000 -302078179.8     25   2780     32 3.04e+08     298     36
    5682  1570001.000 -302078179.8     2

   31209  1570001.000 -259268095.3     25  23109     18 2.61e+08     388     92
   32211  1570001.000 -258586549.8     25  23840     96 2.60e+08      76     95
   33226  1570001.000 -258586549.8     25  24793     48 2.60e+08     157     97
   34234  1570001.000 -258321572.5     25  25505    106 2.60e+08     201    100
g  34898  1560005.000 -258321572.5     26  25938    146 2.60e+08       0    101
g  34900  1540005.000 -258321572.5     27  25938    148 2.60e+08       0    101
b  34901  1490005.000 -258321572.5     28  25938    149 2.60e+08       0    101
Elapsed time (sec): 101, estimated tree completion: 0.02465
Heap usage: 315MB (peak 315MB, 2239KB system)
B&B tree size: 76MB total
 
    Node     BestSoln    BestBound   Sols Active  Depth     Gap     GInf   Time
   35236  1490005.000 -258321572.5     28  25767     48 2.60e+08     176    101
   36236  1490005.000 -257607592.6     28  26461     18 2.59e+08     322    103
   37247  1490005.000 -256665550.9     28  27451    106 2.58e+08  

  192654  1440005.000 -196276535.3     33 155077     89 1.98e+08     110    486
Trying to reduce tree memory usage by 121MB (current usage 303MB, limit 303MB)
Reduced tree memory usage by 135MB (wrote 81MB from 122556 nodes to global file)
  202654  1440005.000 -195876705.3     33 163420     72 1.97e+08     168    511
  212656  1440005.000 -195380207.1     33 171904     37 1.97e+08     175    534
g 219643  1430005.000 -195144363.2     34 177694     98 1.97e+08       0    550
  222656  1430005.000 -194837681.6     34 179567    100 1.96e+08     161    556
  232661  1430005.000 -194413926.9     34 188025    109 1.96e+08      56    578
STOPPING - MAXTIME limit reached (TIMELIMIT=600.00  time=600.00).
STOPPING - MAXTIME limit reached (TIMELIMIT=600.00  time=600.00).
 *** Search unfinished ***    Time:   600.08 Nodes:     241899
Numerical issues encountered:
   Dual failures    :    209 out of    499895 (ratio: 0.0004)
   Singular bases   :     21 out of    336752 (ratio: 0.0001)
Final MIP o